# Free tick level data from zerodha terminal & Automation

There's a way to retrieve historical tick level data through zerodha’s terminal for free. Using Selenium and few python libraries, one can easily get the data automatically. This whole process can be automated.

Blog url - https://medium.com/@algotrading0103/get-stock-market-data-from-zerodha-terminal-for-free-automate-this-process-762be77c472d

### Import & install necessary libraries

In [2]:
pip install pyautogui
pip install pyperclip

In [3]:
import pandas as pd
import json
from time import sleep
import datetime

from selenium import webdriver
from pyotp import TOTP

import pyautogui as pg  #clicking on screen
import pyperclip        #pasting copied text

### Load Login Credentials 

In [4]:
#loading credentials

with open('credentials_zerodha.json','r') as f:
    key_secret = json.load(f)

webdriver_path = key_secret['webdriver_path']
user_id        = key_secret['user_id']
pwd            = key_secret['password']
totp           = key_secret['totp']
url            = key_secret['url']

In [5]:
#loading coordinates

with open('coordinates.json', 'r') as c:
    coord = json.load(c)
    
x_dev_tools     , y_dev_tools     = coord['dev_tools']
x_bottom_symbol , y_bottom_symbol = coord['bottom_symbol']
x_network       , y_network       = coord['network']
x_xhr           , y_xhr           = coord['xhr']
x_refresh       , y_refresh       = coord['refresh']
x_day_chart     , y_day_chart     = coord['day_chart']
x_min_chart     , y_min_chart     = coord['min_chart']
x_minute_url    , y_minute_url    = coord['minute_url']
x_response      , y_response      = coord['response']
x_data          , y_data          = coord['data']

## Per minute data for bankniftymayfut

In [6]:
class Minute():
    
    def __init__(self):
        self.login(webdriver_path, url, user_id, pwd, totp)
        self.inspect(x_dev_tools,y_dev_tools,x_bottom_symbol,y_bottom_symbol,x_network,y_network,x_xhr,y_xhr,x_refresh,y_refresh)
        self.d1 = self.minute_data(x_day_chart,y_day_chart,x_min_chart,y_min_chart,x_minute_url,y_minute_url,x_response,y_response,x_data,y_data)
    
    def login(self, webdriver_path, url, user_id, pwd, totp):
        """
        This function will login to zerodha account with inspect element opened.

        """
        service = webdriver.chrome.service.Service(webdriver_path)
        service.start()

        options = webdriver.ChromeOptions()
        options.add_experimental_option("detach", True)
        options.add_argument("start-maximized")
        options.add_argument("--auto-open-devtools-for-tabs")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)

        options = options.to_capabilities()

        driver = webdriver.Remote(service.service_url, options)
        driver.maximize_window
        driver.get(url)

        sleep(1)

        username = driver.find_element_by_xpath("//input[@type = 'text']")
        username.send_keys(user_id)

        password = driver.find_element_by_xpath("//input[@type = 'password']")
        password.send_keys(pwd)

        driver.find_element_by_xpath("//button[@type = 'submit']").click()

        sleep(1)

        ztotp = driver.find_element_by_xpath("//input[@type = 'text']")
        totp_token = TOTP(totp)
        token = totp_token.now()
        ztotp.send_keys(token)

        driver.find_element_by_xpath("//button[@type = 'submit']").click()

        #calling another function
        self.stock_1(driver)
        
    def stock_1(self, driver):
        """
        This function finds BankNifty future and gets its chart

        """
        #marketwatch
        sleep(1)
        driver.find_element_by_xpath("/html/body/div[1]/div[1]/div/div/div[1]/a[1]/span").click()

        sleep(1)
        #click on banknifty
        driver.find_element_by_xpath('//*[@id="app"]/div[2]/div/div/div/div[2]/div/div[1]/div/div/span[1]/span/span').click()

        sleep(1)
        #click on chart
        driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div/div[2]/ul/li[4]/ul/li[3]/a').click()
    
    def inspect(self, x_dev_tools,y_dev_tools,x_bottom_symbol,y_bottom_symbol,x_network,y_network,x_xhr,y_xhr,x_refresh,y_refresh):
        """
        This function will open developer tools and navigate to network panel.
        Input parameters are the coordinates of buttons on screen.

        """
        #click on inspect menu
        sleep(1)
        pg.click(x_dev_tools, y_dev_tools, interval=1)

        #click on bottom symbol
        pg.click(x_bottom_symbol, y_bottom_symbol, interval=1)

        #click on network panel
        pg.click(x_network, y_network, interval=1)

        #click on Fetch/XHR
        pg.click(x_xhr, y_xhr, interval=1)
        
        #refreshing page
        pg.click(x_refresh, y_refresh)

      
    def minute_data(self,x_day_chart,y_day_chart,x_min_chart,y_min_chart,x_minute_url,y_minute_url,x_response,y_response,x_data,y_data):

        #convert day to minute
        sleep(2)
        pg.click(x_day_chart , y_day_chart, interval=1)

        #select 1 minute
        pg.click(x_min_chart, y_min_chart , interval=1)

        #click on minute url
        pg.click(x_minute_url, y_minute_url , interval=1)

        #click on Response
        pg.click(x_response, y_response , interval=1)

        #click on text
        pg.click(x_data, y_data , interval=1)

        #copying data
        sleep(1)
        pg.hotkey('ctrl', 'a')

        sleep(1)
        pg.hotkey('ctrl', 'c')

        data = pyperclip.paste()
        data = json.loads(data)
        
        self.d1 = self.final_data(data)

        return self.d1
    
    def final_data(self, data):
        candleinfo = data['data']['candles']

        df = pd.DataFrame(candleinfo, columns = ['time','open','high','low','close','volume','openinterest'])
        df['time'] = pd.to_datetime(df.time, format = '%Y-%m-%dT%H:%M:%S')

        date = datetime.datetime.now()        
        if len(str(date.day))==1:
            d = '0'+str(date.day)
        if len(str(date.day))==2:
            d = str(date.day)

        df = df[df.time.astype('str').str.slice(8,10).str.contains(d)]
        df.reset_index(inplace=True, drop=True)
        
        return df

In [7]:
df = Minute()

C:\Users\ganesh\AppData\Local\Temp\ipykernel_15568\4166321570.py:25: DeprecationWarning: desired_capabilities has been deprecated, please pass in an Options object with options kwarg
  driver = webdriver.Remote(service.service_url, options)
C:\Users\ganesh\AppData\Local\Temp\ipykernel_15568\4166321570.py:31: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  username = driver.find_element_by_xpath("//input[@type = 'text']")
C:\Users\ganesh\AppData\Local\Temp\ipykernel_15568\4166321570.py:34: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  password = driver.find_element_by_xpath("//input[@type = 'password']")
C:\Users\ganesh\AppData\Local\Temp\ipykernel_15568\4166321570.py:37: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath("//button[@type = 'submit']").click()
C:\Users\ganesh\AppData\Local\Temp\ipykernel_15568

In [68]:
df.d1

,time,open,high,low,close,volume,openinterest
0,2022-06-02 09:15:00+05:30,35470.95,35508.30,35416.95,35447.95,0,0
1,2022-06-02 09:16:00+05:30,35443.95,35444.05,35416.95,35441.35,0,0
2,2022-06-02 09:17:00+05:30,35437.45,35457.90,35432.35,35440.20,0,0
3,2022-06-02 09:18:00+05:30,35438.85,35452.60,35423.70,35428.70,0,0
4,2022-06-02 09:19:00+05:30,35428.20,35437.10,35410.35,35419.30,0,0
...,...,...,...,...,...,...,...
370,2022-06-02 15:25:00+05:30,35631.55,35640.30,35622.50,35629.00,0,0
371,2022-06-02 15:26:00+05:30,35626.25,35626.25,35604.25,35605.35,0,0
372,2022-06-02 15:27:00+05:30,35605.25,35617.40,35602.65,35614.20,0,0
373,2022-06-02 15:28:00+05:30,35613.45,35617.80,35604.80,35608.35,0,0
